# Assigment 4, comparing distributions

Compare distributions of

1) Trip durations of bikers that ride during the day vs the night  
2) Trip durations of bikers for trips that start in Manhattan vs Brooklyn

In [9]:
import pylab as pl
import pandas as pd
import numpy as np
import os
import json
import scipy.stats

From the previous assigmnment: function to get data

In [6]:
import requests, zipfile,io
from io import StringIO
from zipfile import ZipFile
from urllib.request import urlopen
def getData(datestring):
    zip_file_url=("https://s3.amazonaws.com/tripdata/" + datestring + "-citibike-tripdata.zip")
    r = requests.get(zip_file_url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()
    df = pd.read_csv(z.open(datestring+'-citibike-tripdata.csv'))
    return df

In [7]:
#getting data from two months, January and July, and combining them. 
df1=getData("201501")
df2=getData("201507")
df=df1.append(df2, ignore_index=True)

In [8]:
#confirm there is data from January at the beginning
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1346,1/1/2015 0:01,1/1/2015 0:24,455,1 Ave & E 44 St,40.750020,-73.969053,265,Stanton St & Chrystie St,40.722293,-73.991475,18660,Subscriber,1960.0,2
1,363,1/1/2015 0:02,1/1/2015 0:08,434,9 Ave & W 18 St,40.743174,-74.003664,482,W 15 St & 7 Ave,40.739355,-73.999318,16085,Subscriber,1963.0,1
2,346,1/1/2015 0:04,1/1/2015 0:10,491,E 24 St & Park Ave S,40.740964,-73.986022,505,6 Ave & W 33 St,40.749013,-73.988484,20845,Subscriber,1974.0,1
3,182,1/1/2015 0:04,1/1/2015 0:07,384,Fulton St & Waverly Ave,40.683178,-73.965964,399,Lafayette Ave & St James Pl,40.688515,-73.964763,19610,Subscriber,1969.0,1
4,969,1/1/2015 0:05,1/1/2015 0:21,474,5 Ave & E 29 St,40.745168,-73.986831,432,E 7 St & Avenue A,40.726218,-73.983799,20197,Subscriber,1977.0,1


In [5]:
#confirm that there is data from July at the end...
df.tail()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
1371223,275,7/31/2015 23:59:40,8/1/2015 00:04:15,383,Greenwich Ave & Charles St,40.735238,-74.000271,404,9 Ave & W 14 St,40.740583,-74.005509,22667,Subscriber,1987.0,1
1371224,530,7/31/2015 23:59:41,8/1/2015 00:08:32,455,1 Ave & E 44 St,40.750020,-73.969053,507,E 25 St & 2 Ave,40.739126,-73.979738,20640,Subscriber,1960.0,1
1371225,273,7/31/2015 23:59:42,8/1/2015 00:04:15,383,Greenwich Ave & Charles St,40.735238,-74.000271,404,9 Ave & W 14 St,40.740583,-74.005509,22134,Customer,NaN,0
1371226,1273,7/31/2015 23:59:56,8/1/2015 00:21:09,477,W 41 St & 8 Ave,40.756405,-73.990026,432,E 7 St & Avenue A,40.726218,-73.983799,22861,Subscriber,1986.0,1
1371227,1667,7/31/2015 23:59:59,8/1/2015 00:27:46,2021,W 45 St & 8 Ave,40.759291,-73.988597,401,Allen St & Rivington St,40.720196,-73.989978,16458,Subscriber,1988.0,1


## Part 1, duration at night vs day

** $ H_0 $ ** : The distribution of trip durations during the day is the same as the distribution of trip durations during the night

Test at $ \alpha= 0.05 $

In [17]:
df['hour'] = pd.to_datetime(df['starttime'])

In [18]:
df["hour"]=df["hour"].dt.hour

In [235]:
dfDay=df[['tripduration','hour']]

In [236]:
#define to be between 
dfDay["day"]=(df["hour"]>9) & (df["hour"]<21)

/Users/biancabrusco/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [237]:
dfDay.head()

,tripduration,hour,day
0,1346,0,False
1,363,0,False
2,346,0,False
3,182,0,False
4,969,0,False


In [238]:
dfDay.describe()

,tripduration,hour
count,1.371228e+06,1.371228e+06
mean,9.024175e+02,1.402172e+01
std,7.164578e+03,4.990542e+00
min,6.000000e+01,0.000000e+00
25%,3.910000e+02,1.000000e+01
50%,6.240000e+02,1.500000e+01
75%,1.031000e+03,1.800000e+01
max,5.174309e+06,2.300000e+01


In [239]:
#check if there is any NaN -- there is not. 
dfDay.day.unique()

array([False,  True], dtype=bool)

** KS TEST FOR EQUAL DISTRIBUTION **

In [45]:
ks=scipy.stats.ks_2samp((dfDay.tripduration[dfDay.day == True]),(dfDay.tripduration[dfDay.day == False]))

In [51]:
print(ks)

Ks_2sampResult(statistic=0.063414368899068818, pvalue=0.0)


Recall: we have that under the null hypothesis the two distributions are equal. The test results in a ks-statistic of 0.063, which has corresponding p-value very close to 0. Therefore, at the $\alpha = 0.05$ level we can reject the null hypothesis and conclude that the two distributions differ. 

How to read the ks-statistic value?
In this case we are going to refer to the statistical tests table like the one in the skeleton notebook -- we have that a value above 1.95/sqrt(n)=1.95/1371228=1.42e-06   --  (such as the one we have, of 0.063 )then we have that the p-value is of 0.001, i.e. below our $ \alpha $. Again, we can reject the null hypothesis and conclude that the two distributions are not equal. 

In [57]:
dfDay.shape

(1371228, 3)

In [58]:
1.95/1371228

1.4220829796357716e-06

### Testing for correlation

** $ H_0 $ ** : The correlation between trip duration during Day and trip duration during night is equal to 0. 
Test at $ \alpha= 0.05 $

#### Pearson Test
The way i have previously split the data does not work here because i need to divide the sample into equal sizes of day trips and night trips 

We need to make it so the two are equal. 

In [259]:
len(dfDay[dfDay.day== True])

936461

In [260]:
len(dfDay[dfDay.day==False])

434767

Therefore we see that the group of night rides is smaller than group of day rides. We take a random sample of day rides so that two groups have equal size. 

In [261]:
dfDayNight=dfDay[dfDay.day ==False]

In [262]:
dfDayNight.head()

,tripduration,hour,day
0,1346,0,False
1,363,0,False
2,346,0,False
3,182,0,False
4,969,0,False


In [241]:
dfDayNight.shape

(434767, 3)

In [256]:
#sample so that the two are the same size. Take size of the smallest, i.e. night.
dfDaySame=dfDay[dfDay.day == True].sample(len(dfDayNight))

In [246]:
dfDaySame.head()

,tripduration,hour,day
743673,668,20,True
602957,876,18,True
256878,442,19,True
844261,2017,14,True
9770,819,13,True


In [247]:
dfDayNight.shape

(434767, 3)

In [249]:
prs=scipy.stats.pearsonr(dfDaySame.tripduration,dfDayNight.tripduration)

In [250]:
prs

(-0.00019266535848366787, 0.89891101049431943)

We have a correlation of -0.0002. The Pearson correlation test has a p-value p=0.89, we fail to reject the null hypothesis. Therefore, we **cannot** conclude that the correlation between the two variables is different from 0. 

#### Spearman Test

In [254]:
spr=scipy.stats.spearmanr(dfDaySame.tripduration,dfDayNight.tripduration)

In [255]:
spr

SpearmanrResult(correlation=-0.00038895419310509935, pvalue=0.79759298511349885)

Similarly, we have that a correlatio of -0.004. The p-value of the Spearman test is of p=0.80, so we fail to reject the null hypothesis. We conclud that the correlation is **not** different from 0. 

## Part 2, Manhattan vs Brooklyn

** $ H_0 $ ** : The distribution of trip durations for trips starting in  Manhattan is the same as the distribution of trip durations for trips starting in Brooklyn.  

Test at $ \alpha= 0.05 $

In [12]:
from geopy.geocoders import Nominatim

In [13]:
geolocator=Nominatim()

In [304]:
#TEST=df.head()

In [14]:
df.rename(columns={'start station latitude':'lat'}, inplace=True) 

In [15]:
df.rename(columns={'start station longitude':'lon'}, inplace=True)

In [16]:
df.head()

,tripduration,starttime,stoptime,start station id,start station name,lat,lon,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1346,1/1/2015 0:01,1/1/2015 0:24,455,1 Ave & E 44 St,40.750020,-73.969053,265,Stanton St & Chrystie St,40.722293,-73.991475,18660,Subscriber,1960.0,2
1,363,1/1/2015 0:02,1/1/2015 0:08,434,9 Ave & W 18 St,40.743174,-74.003664,482,W 15 St & 7 Ave,40.739355,-73.999318,16085,Subscriber,1963.0,1
2,346,1/1/2015 0:04,1/1/2015 0:10,491,E 24 St & Park Ave S,40.740964,-73.986022,505,6 Ave & W 33 St,40.749013,-73.988484,20845,Subscriber,1974.0,1
3,182,1/1/2015 0:04,1/1/2015 0:07,384,Fulton St & Waverly Ave,40.683178,-73.965964,399,Lafayette Ave & St James Pl,40.688515,-73.964763,19610,Subscriber,1969.0,1
4,969,1/1/2015 0:05,1/1/2015 0:21,474,5 Ave & E 29 St,40.745168,-73.986831,432,E 7 St & Avenue A,40.726218,-73.983799,20197,Subscriber,1977.0,1


In [17]:
df["loc"]=df["lat"].map(str) + ', ' + df["lon"].map(str)

In [28]:
dfNew=df.head(50)

In [29]:
#df["hood"]= dfNew["loc"].apply(geolocator.reverse,timeout=10)
dfNew["hood"]= dfNew["loc"].apply(geolocator.reverse,timeout=10)

/Users/biancabrusco/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [33]:
#df=df[["tripduration",'hood']]
dfNew=dfNew[["tripduration",'hood']]

In [30]:
#df["hood"]=df["hood"].apply(lambda x: str(x))
dfNew["hood"]=dfNew["hood"].apply(lambda x: str(x))

/Users/biancabrusco/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [31]:
#df["hood"]=df["hood"].apply(lambda x: x.split(",")[3])
dfNew["hood"]=dfNew["hood"].apply(lambda x: x.split(",")[3])

/Users/biancabrusco/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [35]:
dfNew.head()

,tripduration,hood
0,1346,Manhattan Community Board 6
1,363,Manhattan Community Board 4
2,346,Manhattan Community Board 5
3,182,BK
4,969,Manhattan Community Board 5


In [56]:
def hoodsimple(x="hood"):
    if x.startswith(" Manhattan"):
        hood="Manhattan"
        return hood
    elif x.startswith(" BK") | x.startswith(" Kings County"):
        hood="Brooklyn"
        return hood

In [57]:
#df["hoodSimple"]=df["hood"].apply(hoodsimple)
dfNew["hoodSimple"]=dfNew["hood"].apply(hoodsimple)

/Users/biancabrusco/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [68]:
#theoreticall can run all of this with full dataset although there will be more hoods to add to function
dfNew.head()

,tripduration,hood,hoodSimple
0,1346,Manhattan Community Board 6,Manhattan
1,363,Manhattan Community Board 4,Manhattan
2,346,Manhattan Community Board 5,Manhattan
3,182,BK,Brooklyn
4,969,Manhattan Community Board 5,Manhattan


we get till separating to neighbourhoods. Then follow pretty much same as example with Day Night


**KS TEST**

In [62]:
prs2=scipy.stats.pearsonr(dfNew.hoodSimple == "Manhattan", dfNew.hoodSimple == "Brooklyn")

In [63]:
prs2

(-0.81124574459263454, 9.0882242010498267e-13)

With this p value we reject the null hypothesis!!! Two distributions are different!!

H0: is correlation statistically different than 0 between Manhattan and Brooklyn?

**Spearman Test**

In [64]:
spr2=scipy.stats.spearmanr(dfNew.hoodSimple == "Manhattan", dfNew.hoodSimple == "Brooklyn")

In [65]:
spr2

SpearmanrResult(correlation=-0.81124574459263454, pvalue=9.08822420104986e-13)

No -- p value above 0.05 means correlation is not statistically different than 0.